# Getting started - level 3

In this tutorial we will explore a little bit more advanced example of a program that require some configuration, requirements setup, etc. 

Again we will start with writing code for our program and saving it to `./source_files/gs_level_3.py` file.
This time it will be running estimator as parallel functions and saving results to shared state. 

```python
# source_files/gs_level_3.py

from qiskit import QuantumCircuit
from qiskit.circuit.random import random_circuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator

from quantum_serverless import QuantumServerless, run_qiskit_remote, get, put
from quantum_serverless.core.state import RedisStateHandler

# 1. let's annotate out function to convert it
# to function that can be executed remotely
# using `run_qiskit_remote` decorator
@run_qiskit_remote()
def my_function(circuit: QuantumCircuit, obs: SparsePauliOp):
    return Estimator().run([circuit], [obs]).result().values


# 2. Next let's create out serverless object to control
# where our remote function will be executed
serverless = QuantumServerless()

# 2.1 (Optional) state handler to write/read results in/out of job
state_handler = RedisStateHandler("redis", 6379)

circuits = [random_circuit(2, 2) for _ in range(3)]

# 3. create serverless context
with serverless.context():
    # 4. let's put some shared objects into remote storage that will be shared among all executions
    obs_ref = put(SparsePauliOp(["ZZ"]))

    # 4. run our function and get back reference to it
    # as now our function it remote one
    function_reference = my_function(circuits[0], obs_ref)

    # 4.1 or we can run N of them in parallel (for all circuits)
    function_references = [my_function(circ, obs_ref) for circ in circuits]

    # 5. to get results back from reference
    # we need to call `get` on function reference
    single_result = get(function_reference)
    parallel_result = get(function_references)
    print("Single execution:", single_result)
    print("N parallel executions:", parallel_result)

    # 5.1 (Optional) write results to state.
    state_handler.set("result", {
        "status": "ok",
        "single": single_result.tolist(),
        "parallel_result": [entry.tolist() for entry in parallel_result]
    })
```

As you can see we move to advanced section of using serverless. 

Here we are using `run_qiskit_remote` decorator to convert our function to parallel one. 
With that `my_function` is converted into remote call (as a result you will be getting function pointer) and in order to fetch results of this function we need to call `get` function.

Moreover, we are using `RedisStateHandler` in order to save results into state storage, so we can retrieve it later after program execution.

Next we need to run this program. For that we need to import necessary modules and configure QuantumServerless client. We are doing so by providing name and host for deployed infrastructure.

In [1]:
from quantum_serverless import QuantumServerless, Program
from quantum_serverless.core.state import RedisStateHandler

In [2]:
serverless = QuantumServerless({
    "providers": [{
        "name": "docker",
        "compute_resource": {
            "name": "docker",
            "host": "localhost",
        }
    }]
})
serverless

<QuantumServerless | providers [local, docker]>

We will create instance of state handler to fetch results from program after execution

In [3]:
state_handler = RedisStateHandler("localhost", 6379)
state_handler

<RedisStateHandler | localhost:6379>

Run program

In [4]:
program = Program(
    name="Advanced program",
    entrypoint="gs_level_3.py",
    working_dir="./source_files/"
)

job = serverless.run_program(program)
job

<Job | fqs_fcee8245-5088-411f-8b0f-e214a6f5e765>

In [9]:
job.status()

<JobStatus.SUCCEEDED: 'SUCCEEDED'>

In [10]:
print(job.logs())

Single execution: [1.]
N parallel executions: [array([1.]), array([0.]), array([-1.])]



With `state_handler` as can fetch results that we wrote inside the program.

In [11]:
state_handler.get("result")

{'status': 'ok', 'single': [1.0], 'parallel_result': [[1.0], [0.0], [-1.0]]}